In [11]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from tensorflow.python.client import device_lib
# Plotting samples
from matplotlib import pyplot as plt
# Visualizing the model of the NN
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
# from keras.callbacks import History
%matplotlib inline

device_lib.list_local_devices() 

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10360920341669371029, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 104857600
 locality {
   bus_id: 1
 }
 incarnation: 6932900788083646873
 physical_device_desc: "device: 0, name: GeForce GTX 780, pci bus id: 0000:02:00.0"]

In [ ]:
def get_shuffled_splitted_data(path):
    df = pd.read_csv(path)

    # Shuffle and split data
    X_train, X_test, X_val = np.split(df.sample(frac=1), [int(.7*len(df)), int(.9*len(df))])
    
    # Pop labels and transform them to vectors
    y_train, y_test, y_val = X_train.pop("label"), X_test.pop("label"), X_val.pop("label")
    y_train, y_test, y_val = y_train.values.reshape((-1, 1)), y_test.values.reshape((-1, 1)), y_val.values.reshape((-1, 1))
    
    # Reshape the features for CNN
    X_train = X_train.as_matrix().reshape(X_train.shape[0], 1, 124, 124)
    X_test = X_test.as_matrix().reshape(X_test.shape[0], 1, 124, 124)
    X_val = X_val.as_matrix().reshape(X_val.shape[0], 1, 124, 124)
    
    # Norm data
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_val = X_val.astype('float32')
    X_train /= 255
    X_test /= 255
    X_val /= 255
    
    # Convert labels to categorical values
    y_train = keras.utils.to_categorical(y_train, 2)
    y_test = keras.utils.to_categorical(y_test, 2)
    y_val = keras.utils.to_categorical(y_val, 2)
    
    return X_train, y_train, X_test, y_test, X_val, y_val
    
X_train, y_train, X_test, y_test, X_val, y_val = get_shuffled_splitted_data('data/subset-4-All.txt.csv')

In [ ]:
with tf.device('/cpu:0'):

    import keras
    from keras.preprocessing.image import ImageDataGenerator
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Activation, Flatten
    from keras.layers import Conv2D, MaxPooling2D
    from keras import backend as K
    from keras.losses import categorical_crossentropy
    from keras.optimizers import Adadelta
    K.set_image_dim_ordering('th')

    batch_size = 32
    num_classes = 2
    epochs = 5
    history = keras.callbacks.History()  # For storing the validation loss values

    # The data, shuffled and split between train and test sets:
    print('X_train shape:', X_train.shape)
    print(X_val.shape[0], 'validation samples')

    # TODO: Evaluate if using leaky relu, elu or MaxOut is better than just relu
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(1, 124, 124)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])

    # TODO: Evaluate if centering image data is better (https://keras.io/preprocessing/image/)
    hist = model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(X_test, y_test),
              callbacks=[history])

    test_score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', test_score[0])
    print('Test accuracy:', test_score[1])

    val_score = model.evaluate(X_val, y_val, verbose=0)
    print('Val loss:', val_score[0])
    print('Val accuracy:', val_score[1])

X_train shape: (14001, 1, 124, 124)
2001 validation samples
Train on 14001 samples, validate on 4000 samples
Epoch 1/5
14001/14001 [==============================] - 1144s - loss: 0.0159 - acc: 0.9944 - val_loss: 5.8185e-05 - val_acc: 1.0000
Epoch 2/5
 8160/14001 [================>.............] - ETA: 413s - loss: 7.4513e-05 - acc: 1.0000

In [ ]:
val_score = model.evaluate(X_val, y_val, verbose=0)
print('Val loss:', val_score[0])
print('Val accuracy:', val_score[1])

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 120, 120)      832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 60, 60)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 56, 56)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 28, 28)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 28, 28)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               6422656   
__________

In [ ]:
plt.plot(hist.history['loss'])

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
y_test_pred = model.predict(X_test, batch_size=32, verbose=0)
y_test_pred = np.round(y_test_pred).astype(int)

def plot_sample(ax, sample, title):
    # The first line contains 65000 values for any reason
    img = sample.reshape(124, 124)[1:, 1:]
    ax.imshow(img, cmap='gray',  interpolation='nearest')
    ax.axis('off')
    ax.set_title(title)

def has_tumor(one_hot_vector):
    return one_hot_vector.argmax()
    
def plot_samples(count, samples, labels, predicted, main_title):
    # Shuffle datapoints
    idx = np.random.choice(np.arange(samples.shape[0]), count, replace=False)
    samples, labels, predicted = (samples[idx], labels[idx], predicted[idx])
    cols = 4
    rows = count // cols
    assert rows * cols == count, 'Number of samples must be a multiple of 4'
    fig, axes = plt.subplots(rows, cols, figsize=(16, rows * 4))
    for i, ax in enumerate(axes.flat):
        plot_sample(ax, samples[i], '#{}, Tumor: {}, Predicted: {}'.format(
            idx[i], has_tumor(labels[i]), has_tumor(predicted[i])))
    fig.suptitle(main_title)

# Always the same results
np.random.seed(0)
plot_samples(4, X_test, y_test, y_test_pred, 'Testing set')
no_tumors = y_test.argmax(axis=1) == 0
plot_samples(4, X_test[no_tumors], y_test[no_tumors],
             y_test_pred[no_tumors], 'Testing set - No tumor')
plot_samples(4, X_test[no_tumors == False], y_test[no_tumors == False],
             y_test_pred[no_tumors == False], 'Testing set - Tumor')

keras.utils.plot_model(model, show_shapes=True, to_file='model-Small-tk.png')
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))